# Projeto 1 - Ciência dos Dados

Nome: __Alexandre Antar__

Nome: __Gabriel de Araujo Alves__

Nome: __Renato Fanfrachini Falcao__

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
!pip install emoji wordcloud
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI
import emoji
import functools
import operator
import nltk



In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\projeto1-Cdados#\Project1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'brahma.xlsx'

In [5]:
#base de treinamento
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Relevancia
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,0
1,@moraesbrunita pelo visto na mamadeira já era ...,0
2,@purosucodosurto o cara não tem um pau tem um ...,0
3,gente vcs que tão em home office a mais tempo ...,1
4,rt @luciusnaweb: eu depois de dois latões de b...,0


In [8]:
#base de teste
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,@elissanmanoel tão costurando brahma ? hahaha ...,0
1,rt @nayaralucas11: era só uma praia e brahma t...,0
2,arrumando meu guarda roupa eis que encontro um...,0
3,desceu foi whisky misturado c brahma kkkkkkkkk...,0
4,rt @carolaguiarcm: hj eu só queria estar numa ...,0


In [9]:
#transformando variavel Relevancia em categórica
train['Relevancia'] = train['Relevancia'].astype('category')

train['Relevancia'].cat.categories = ['Relevante negativa', 'Não relevante', 'Relevante positiva']
train.head(6)

,Treinamento,Relevancia
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,Não relevante
1,@moraesbrunita pelo visto na mamadeira já era ...,Não relevante
2,@purosucodosurto o cara não tem um pau tem um ...,Não relevante
3,gente vcs que tão em home office a mais tempo ...,Relevante positiva
4,rt @luciusnaweb: eu depois de dois latões de b...,Não relevante
5,"acho q vou beber uma hj, desde domingo sem beb...",Relevante negativa


In [37]:
#separando o data frame por relevância do tweet
nao_relevante=train[train.Relevancia=='Não relevante']
relevante_positivo=train[train.Relevancia=='Relevante positiva']
relevante_negativo=train[train.Relevancia=='Relevante negativa']
classifica=[nao_relevante,relevante_positivo,relevante_negativo]




In [12]:
#uma lista de strings para aferir a probabilidade posteriormente,não necessariamente será usada
classifica_nao_relevante=str(list(classifica[0].Treinamento.values))
classifica_positivo=str(list(classifica[1].Treinamento.values))
classifica_negativo=str(list(classifica[2].Treinamento.values))



___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi a cerveja Brahma, uma marca nacional e bem popular de cerveja e a classificação dos tweets foram feitos da seguinte forma:

* -1 : Relevante negativo -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma negativa, seja com reclamações acerca do produto ou associando a marca à alguma experiência ruim.
* 0 : Irrelevante -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma neutra, sem reclamações ou elogios ao produto ou que não se referiram à cerveja.
* 1 : Relevante positivo -> Essa classificação se refere aos tweets que mecionavam o a marca de cerveja de forma positiva, seja com elogios acerca do produto ou associando a marca à alguma experiência boa.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [13]:
#limpeza de mensagens em relação a pontuação
def cleanup_tweet(text):
    punctuation = '[\-/!.:?;,''"@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(' +', ' ', text_subbed)
    text_subbed = emoji.get_emoji_regexp().split(text_subbed)
    separate = [text.split() for text in text_subbed]
    text_subbed = functools.reduce(operator.concat, separate)
    return text_subbed


In [19]:
#remoção de outros items que não contarão para as relevâncias
def remove_outros(items):
    i=0
    while(i<len(items)):
        # Limpa '\n'
        items[i] = items[i].lower().replace('[\n]', '')
        
        # Remove 'rt' de retweets
        if items[i][0:2] == 'rt':
            items[i] = items[i][3:]
        
        # Remove links
        elif items[i][:4] == 'http':
            del items[i]
        i+=1
            
        
    return items

In [20]:
# Remove 'stopwords' (preposições, artigos etc.) dos tweets
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.append('')

# Define função que aplica remoção
def remove_stopwords(lista):
    filtrada = lista[:]     
    for palavra in lista: 
          if palavra in stopwords: 
                filtrada.remove(palavra)               
    return filtrada

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
#twt=cada tweet
#aplica todas as funções anteriores para limpar as frases
def limpos(classificador):
    lis=[]
    for twt in classificador.Treinamento:
        lis.append(remove_stopwords(remove_outros(cleanup_tweet(twt))))
    return lis


In [39]:
# Aplica limpeza nas séries de cada classificação
limpeza_tweets = [limpos(classifica[0]), limpos(classifica[1]),limpos(classifica[2])]

# Limpa dados de treino
limpeza = limpos(train)

# Aplica listagem das palavras dos dados de treino
for i in range(len(limpeza)):
    limpeza[i] = ', '.join(limpeza[i])

# Cria coluna com tweets já limpos
train['Treinamento_limpo'] = limpeza



train.head()



,Treinamento,Relevancia,Treinamento_limpo
0,rt @sophiaecris: @marianaa2mari foi vcs crlh !...,Não relevante,"sophiaecris, marianaa2mari, vcs, crlh, boa, br..."
1,@moraesbrunita pelo visto na mamadeira já era ...,Não relevante,"moraesbrunita, visto, mamadeira, boa, brahma, ..."
2,@purosucodosurto o cara não tem um pau tem um ...,Não relevante,"purosucodosurto, cara, pau, latão, brahma"
3,gente vcs que tão em home office a mais tempo ...,Relevante positiva,"gente, vcs, tão, home, office, tempo, ok, mand..."
4,rt @luciusnaweb: eu depois de dois latões de b...,Não relevante,"luciusnaweb, dois, latões, brahma, bem, assim,..."


###2° ETAPA


 

Agora com os dados limpos ,o objetivo e ensinar o classificador de acordo com as porbabilidades de cada palavra apresentada para cada série de relevância

In [40]:
#value_counts em relação aos tweets irrelevantes,assim o pd_series afere todas as probabilidades por palavra nessa série
n_rel=train[train.Relevancia=='Não relevante']
n_rel1=n_rel.Treinamento_limpo
lista1=[]
for tweet in n_rel1:
    for palavra in tweet.split(','):
        lista1.append(palavra)
serie_n_rel=pd.Series(lista1).value_counts(True)
tot_n_rel=len(serie_n_rel)
tot_n_rel





1327

In [41]:
#value_counts em relação aos tweets relevantes positivos,assim o pd_series afere todas as probabilidades por palavra nessa série
rel_pos=train[train.Relevancia=='Relevante positiva']
rel_pos1=rel_pos.Treinamento_limpo
lista2=[]
for tweet in rel_pos1:
    for palavra in tweet.split(','):
        lista2.append(palavra)
serie_pos=pd.Series(lista2).value_counts(True)
tot_pos=len(serie_pos)
tot_pos


1108

In [52]:
##value_counts em relação aos tweets relevantes negativos,assim o pd_series afere todas as probabilidades por palavra nessa série
rel_neg=train[train.Relevancia=='Relevante negativa']
rel_neg1=rel_neg.Treinamento_limpo
lista3=[]
for tweet in rel_neg1:
    for palavra in tweet.split(','):
        lista3.append(palavra)
serie_neg=pd.Series(lista3).value_counts(True)
tot_neg=len(serie_neg)
tot_neg
serie_neg




 brahma       0.097166
brahma        0.024291
 🤤            0.020243
 dor          0.020243
 dia          0.016194
                ...   
 geladinha    0.004049
 ñ            0.004049
 hj           0.004049
 estágio      0.004049
 cacete       0.004049
Length: 170, dtype: float64

In [54]:
lista4=[list(serie_n_rel.index),list(serie_pos.index),list(serie_neg.index)]
lista5=[list(serie_n_rel.values),list(serie_pos.values),list(serie_neg.values)]

probs=[]
for i in range(len(lista4)):
    dicio = {}
    for j in range(len(lista4[i])):
        dicio[lista4[i][j]]=lista5[i][j]
    probs.append(pd.Series(dicio))




        


# # probs.append(pd.Series(dicio))
# # lista5[0]

In [44]:
#lista para as palavras não repetidas
def repetidas(lista1, lista2,lista3):
    qtd = []
    for tweet in lista1:
        for palavra in tweet:
            if palavra not in qtd:
                qtd.append(palavra)                
    for tweet in lista2:
        for palavra in tweet:
            if palavra not in qtd:
                qtd.append(palavra)
    for tweet in lista3:
        for palavra in tweet:
            if palavra not in qtd:
                qtd.append(palavra)
    return qtd

In [45]:
#retorna o total de palavras não repetidas em relacao as três classificações
tol = (serie_n_rel + serie_pos + serie_neg).index.nunique()
tol

2169

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [46]:
def limpando(tweet):
    limpo = remove_stopwords(remove_outros(cleanup_tweet(str(tweet))))
    return limpo

test_tweets = []
for tweet in test['Teste']:
    test_tweets.append(limpando(tweet))

# Cria coluna de pré processados nos dados de teste
series_test = pd.Series(test_tweets)
test['Pré Processado'] = series_test

#substitui a relevancia de numeros para categorias
test['Relevancia'] = test['Relevancia'].astype('category')

test['Relevancia'].cat.categories = ['Relevante negativa', 'Não relevante', 'Relevante positiva']
test.head(6)

,Teste,Relevancia,Pré Processado,Naive Bayes
0,@elissanmanoel tão costurando brahma ? hahaha ...,Não relevante,"[elissanmanoel, tão, costurando, brahma, hahah...",Relevante negativa
1,rt @nayaralucas11: era só uma praia e brahma t...,Não relevante,"[nayaralucas11, praia, brahma, trincando]",Relevante negativa
2,arrumando meu guarda roupa eis que encontro um...,Não relevante,"[arrumando, guarda, roupa, eis, encontro, ling...",Relevante negativa
3,desceu foi whisky misturado c brahma kkkkkkkkk...,Não relevante,"[desceu, whisky, misturado, c, brahma, kkkkkkk...",Relevante negativa
4,rt @carolaguiarcm: hj eu só queria estar numa ...,Não relevante,"[carolaguiarcm, hj, queria, estar, mesa, skol,...",Relevante negativa
5,vó beber brahma fodaze,Não relevante,"[vó, beber, brahma, fodaze]",Relevante negativa


In [47]:
# #limpeza dos tweets do teste
# def limpando(tweet):
#     limpo = remove_stopwords(remove_outros(cleanup_tweet(str(tweet))))
#     return limpo
# #lista para os tweets
# test_tweets = []
# for tweet in test['Teste']:
#     test_tweets.append(limpando(tweet))

# # Cria coluna de tweets limpos nos dados de teste
# series_test = pd.Series(test_tweets)
# test['Pré Processado'] = series_test
# test

In [33]:
def compara(frase):
    """
    Essa função determina a taxa relativa referente a cada palavra de uma frase
    para o cálculo da probabilidade condicional de Naive-Bayes, fazendo a
    comparação entre a probabilidade ser relevante ou irrelevante e retorna
    a resposta (relevante ou irrelevante) com base na desigualdade.
    """
    prob_nao_relevante=1
    prob_relevante_positivo=1
    prob_relevante_negativo=1

    for palavra in frase:
        if (palavra not in probs[0].index):
            prob_nao_relevante *=1/(tot_n_rel+tol)
        else: 
            prob_nao_relevante *= probs[0][palavra]
        

    for palavra in frase:
        if (palavra not in probs[1].index):
            prob_relevante_positivo *=1/(tot_pos+tol)
        else:
            prob_relevante_positivo *= probs[1][palavra]
    for palavra in frase:
        if (palavra not in probs[2].index):
            prob_relevante_negativo *=1/(tot_neg+tol)
        else:
            prob_relevante_negativo *= probs[2][palavra]
    
    list_prob = [prob_nao_relevante,prob_relevante_positivo,prob_relevante_negativo]
    index = list_prob.index(max(list_prob))
    
            
    if index == 0:
        return 'Não Relevante'
    elif index==1:
        return 'Relevante positiva'
    elif index==2:
        return 'Relevante negativa'


In [56]:
# def conclusao(coluna, linha):
#     """
#     Aplica função 'conclusão' para linha n da coluna do dataframe
#     """
#     return compara(coluna[linha])

# Para cada tweet de teste, aplica o modelo e compara com o rótulo dado
classificacao_modelo = []
for i in range(0, len(test['Teste'])):
    classificacao_modelo.append(compara(test['Pré Processado'][i]))
    
    

# Cria coluna nos dados de teste com os resultados
test["Naive Bayes"] = classificacao_modelo

# Exibe comparativo dos resultados do modelo com os rótulos (em porcentagem)
ct = pd.crosstab(test['Relevancia'], test['Naive Bayes'], normalize = True).round(4)*100
ct



Naive Bayes,Não Relevante,Relevante negativa,Relevante positiva
Relevancia,,,
Relevante negativa,0.00,2.75,0.00
Não relevante,0.31,50.46,0.31
Relevante positiva,0.00,45.26,0.92


In [36]:
serie_0_test = test[test["Relevancia"] == 'Não relevante']
serie_1_test = test[test["Relevancia"] =='Relevante positiva']
serie_2_test=test[test['Relevancia']=='Relevante negativa']

# Valores iniciais das quantidades de cada caso
verdadeiros_positivos = 0
falsos_positivos = 0
verdadeiros_negativos = 0
falsos_negativos = 0

# Calcula a quantidade de tweets de cada um dos casos acima
for i in range(len(test['Relevancia'])):
    if test['Relevancia'][i] == 'Não relevante' and test['Naive Bayes'][i] == 'Não relevante':
        verdadeiros_positivos += 1
    elif test['Relevancia'][i] == 'Não relevante' and test['Naive Bayes'][i] =!'Não relevante' :
        falsos_positivos += 1
    elif test['Relevancia'][i] == 0 and test['Naive Bayes'][i] == 0:
        verdadeiros_negativos += 1
    elif test['Relevancia'][i] == 1 and test['Naive Bayes'][i] == 0:
        falsos_negativos += 1

# Retorna probabilidade para cada caso
print("Contagem\t\tProbabilidade")
print("----------------------------------------")
print("%s:\t\t%f" % ('Verdadeiros Positivos', verdadeiros_positivos/len(serie_1_test['Relevancia'])))
print("%s:\t\t%f" % ('Falsos Positivos', falsos_positivos/len(serie_0_test['Relevancia'])))
print("%s:\t\t%f" % ('Verdadeiros Negativos', verdadeiros_negativos/len(serie_0_test['Relevancia'])))
print("%s:\t\t%f" % ('Falsos Negativos', falsos_negativos/len(serie_1_test['Relevancia'])))

SyntaxError: invalid syntax (<ipython-input-36-992ed1b6a8de>, line 15)

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [36]:
# # Definindo algumas funções necessárias ao processo

# def laplace(quantidade_na_serie, total_serie, total):
#     return (quantidade_na_serie + 1)/(total_serie + total)

# def vezes_que_aparece(palavra, serie):
#     """
#     Essa função recebe uma determinada palavra e a série de palavras, 
#     retornando a quantidade de vezes que ela aparece na série
#     """
#     quant = 0
#     for tweet in serie:
#         quant += tweet.count(palavra)
#     return quant

# def total_na_serie(serie):
#     """
#     Essa função retorna o total de palavras da série
#     """
#     quant = 0
#     for tweet in serie:
#         quant += len(tweet)
#     return quant

# def lista_palavras(lista1, lista2,lista3):
#     """
#     Essa função retorna as palavras (sem repetições) presentes
#     em cada uma das listas de entrada
#     """
#     quant = []
#     for tweet in lista1:
#         for palavra in tweet:
#             if palavra not in quant:
#                 quant.append(palavra)                
#     for tweet in lista2:
#         for palavra in tweet:
#             if palavra not in quant:
#                 quant.append(palavra)
#     for tweet in lista3:
#             if palavra not in quant:
#                 quant.append(palavra)
#     return quant
